In [7]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

from G_A_layer_tuning import run_G_A
from fine_tuning import *

In [8]:
def load_urbansound8k(load_path):
    '''
    loaded = np.load('파일명.npz', allow_pickle=True)
    (128, 173, 1)
    '''

    train_path = os.path.join(load_path, 'fold1train.npy')
    test_path = os.path.join(load_path, 'fold1test.npy')

    train_data = np.load(train_path, allow_pickle=True)
    test_data = np.load(test_path, allow_pickle=True)

    x_train = train_data[:, -1]
    y_train = train_data[:, 1]
    x_test = test_data[:, -1]
    y_test = test_data[:, 1]

    x_train = np.stack(x_train, axis=0)
    x_test = np.stack(x_test, axis=0)

    x_train = x_train[...,np.newaxis]
    x_test = x_test[...,np.newaxis]
    y_train = y_train.reshape((-1,))
    y_test = y_test.reshape((-1,))

    # 배열의 원소 개수만큼 인덱스 배열을 만든 후
    # 무작위로 뒤섞어 줍니다.
    idx_train = np.arange(x_train.shape[0])
    idx_test = np.arange(x_test.shape[0])
    # print(idx)
    np.random.shuffle(idx_train)
    np.random.shuffle(idx_test)

    x_train_shuffle = x_train[idx_train]
    y_train_shuffle = y_train[idx_train]
    x_test_shuffle = x_test[idx_test]
    y_test_shuffle = y_test[idx_test]

    # TensorFlow에서 읽을 수 있는 np.astype으로 변경
    x_train = np.asarray(x_train_shuffle).astype(np.float64)
    y_train = np.asarray(y_train_shuffle).astype(np.int64)
    x_test = np.asarray(x_test_shuffle).astype(np.float64)
    y_test = np.asarray(y_test_shuffle).astype(np.int64)

    print('Train/Test Dataset Shape')
    print('x_train shape is ', x_train.shape)
    print('y_train shape is ', y_train.shape)
    print('x_test shape is ', x_test.shape)
    print('y_test shape is ', y_test.shape)

    return x_train, y_train, x_test, y_test

In [9]:
def load_hospitalalarmsound(load_path):
    '''
    loaded = np.load('파일명.npz')
    loaded['spectrogram']
    loaded['Mel_spectrogram']
    loaded['Log_Mel_spectrogram']
    loaded['mfcc']
    loaded['delta_mfcc']
    '''

    for idx, file in enumerate(os.listdir(load_path)):
        filepath = os.path.join(load_path, file)
        loaded = np.load(filepath)
        # print('class number :',idx, file[:-4])
        x = loaded['Log_Mel_spectrogram']
        y = np.zeros((x.shape[0],1), dtype=np.int8)
        y = y + idx
        # print(x.shape)
        # print(y.shape)
        ratio = round(x.shape[0]*0.2)   # 각 클래스별 나누는 비율
        flag = x.shape[0]-ratio

        if idx == 0:
            x_train = x[:flag]
            y_train = y[:flag]
            x_test = x[flag:]
            y_test = y[flag:]
        else:
            x_train = np.vstack((x_train, x[:flag]))
            y_train = np.vstack((y_train, y[:flag]))
            x_test = np.vstack((x_test, x[flag:]))
            y_test = np.vstack((y_test, y[flag:]))

    x_train = x_train[...,np.newaxis]
    x_test = x_test[...,np.newaxis]
    y_train = y_train.reshape((-1,))
    y_test = y_test.reshape((-1,))

    # 배열의 원소 개수만큼 인덱스 배열을 만든 후 
    # 무작위로 뒤섞어 줍니다. 
    idx_train = np.arange(x_train.shape[0])
    idx_test = np.arange(x_test.shape[0])
    # print(idx)
    np.random.shuffle(idx_train)
    np.random.shuffle(idx_test)

    x_train_shuffle = x_train[idx_train]
    y_train_shuffle = y_train[idx_train]
    x_test_shuffle = x_test[idx_test]
    y_test_shuffle = y_test[idx_test]

    # TensorFlow에서 읽을 수 있는 np.astype으로 변경
    x_train = np.asarray(x_train_shuffle).astype(np.float64)
    y_train = np.asarray(y_train_shuffle).astype(np.int64)
    x_test = np.asarray(x_test_shuffle).astype(np.float64)
    y_test = np.asarray(y_test_shuffle).astype(np.int64)

    print('Train/Test Dataset Shape')
    print('x_train shape is ', x_train.shape)
    print('y_train shape is ', y_train.shape)
    print('x_test shape is ', x_test.shape)
    print('y_test shape is ', y_test.shape)

    return x_train, y_train, x_test, y_test

In [ ]:
# Parameters for G.A.
num_of_class = 8            
freezing_layer_flag = 0
type = 'vgg'

N_POPULATION = 4
N_BEST = 2
N_CHILDREN = 2
PROB_MUTATION = 0.04

lr = 0.0001
epoch = 4
batch_size = 64

In [ ]:
# Load dataset for hospitalAlarmsound
load_path = 'D:\GH\Audio\dataset\preprocess_data\hospital_alarm\zero_pad_preprocess'      # HospitalAlarmSound : num_of_class=8
save_path = 'D:\\GH\\Audio\\GA\\test\\pickle_data\\HospitalAlarmSound\\0823_vgg'
dataset = load_hospitalalarmsound(load_path)

# Load dataset for urbansound8k
# load_path = 'D:\GH\Audio\dataset\\UrbanSound8K\\audio\\np_data'                           # UrbanSound8K : num_of_class=10
# save_path = 'D:\\GH\\Audio\\GA\\test\\pickle_data\\UrbanSound8K\\0824_vgg'
# dataset = load_urbansound8k(load_path)

x_train, y_train, x_test, y_test = dataset
input_shape = (x_train.shape[1], x_train.shape[2])

In [ ]:
run_G_A(x_train, y_train, x_test, y_test, epoch, batch_size, num_of_class, 
    freezing_layer_flag, type, save_path, 5, N_POPULATION, PROB_MUTATION, lr, False)

G_A_best_bool_arr = load_best_picklefile(save_path)[1]

In [ ]:
# Parameters for test
type ='vgg'
freezing_layer_flag = 0
num_of_class = 8
learning_rate = 0.0001
one_hot_encoding = False
epochs=30
batch_size=32
checkpoint_path='model_checkpoints_best/checkpoint'

In [ ]:
Finetuned_vgg = Finetuing_Pretrained_Model(G_A_best_bool_arr, input_shape, freezing_layer_flag, type)
model = Finetuned_vgg.forward(num_of_class, learning_rate, one_hot_encoding)
history = Finetuned_vgg.train_model(model, x_train, y_train, x_test, y_test, epochs, batch_size, checkpoint_path)

accuracy_result(history)
loss_result(history)

loaded_vgg = Finetuing_Pretrained_Model(None, input_shape, freezing_layer_flag, type)
model = loaded_vgg.forward(num_of_class, learning_rate, one_hot_encoding)
model = loaded_vgg.load_model(model, checkpoint_path)
loss, acc = loaded_vgg.eval_model(model, x_test, y_test, verbose=2)